In [7]:
import pandas as pd
import glob
import seaborn as sns

In [4]:
df = pd.concat([pd.read_csv(f) for f in glob.glob('data/mpc/*.csv')], ignore_index = True)

In [6]:
df

,Unnamed: 0,n,l,grad
0,0,4,80,3.986812e-01
1,0,4,80,-5.563134e-01
2,0,4,80,-2.262908e-01
3,0,4,80,1.700816e-02
4,0,4,80,6.167716e-01
...,...,...,...,...
56405,18,4,180,-6.294157e-01
56406,18,4,180,-3.919271e-01
56407,18,4,180,2.627292e-01
56408,18,4,180,2.775558e-09


In [9]:
df.groupby(by=['n', 'l']).mean()

Unnamed: 0      grad
n l                        
4 10     8.160000  0.037947
  20     9.225806 -0.026268
  30     9.580645 -0.010227
  40     9.809524 -0.000651
  50     9.000000  0.002982
  70     7.370370  0.007717
  80     9.700000 -0.001946
  90     8.400000 -0.000193
  100    9.000000 -0.002421
  110    9.967742  0.000089
  120    7.870968  0.000982
  130   10.166667  0.007093
  140    9.120000  0.001319
  150    8.322581  0.002784
  160   13.500000 -0.000849
  170    7.939394  0.005361
  180    8.774194  0.002124
  190   12.500000  0.004626
6 10     9.000000  0.005395
  20     9.000000 -0.004571
  90     6.000000 -0.005771
  100   15.500000  0.002231